In [212]:
import metapy

In [ ]:
metapy.classify.Winnow

In [205]:
import requests
import lxml.html
import pandas as pd
import numpy as np
from time import sleep
import xml.etree.ElementTree as ET

In [8]:
action_link = 'https://www.imdb.com/search/title?genres=action&explore=title_type,genres&title_type=movie&ref_=adv_explore_rhs'
r = requests.get(action_link)

In [9]:
tree = lxml.html.fromstring(r.content)

In [10]:
# get overall ratings for each movie
ratings = tree.xpath('//*[@class="inline-block ratings-imdb-rating"]/@data-value')

In [11]:
len(ratings)

39

In [297]:
titles = tree.xpath('//div/div[@class ="ratings-bar"]/preceding-sibling::h3/a[contains(@href,"/title/")]')           

In [319]:
movie_title = []
for title in titles:
    movie_title.append(title.xpath('text()')[0])

In [321]:
movie_title = movie_title[:9]+movie_title[10:]

In [324]:
'Stuber'in movie_title

False

In [12]:
movies = tree.xpath('//div/div[@class ="ratings-bar"]/preceding-sibling::h3/a[contains(@href,"/title/")]/@href')
ID = [movie[7:-1] for movie in movies]  # unique ID for each movie
links = ['https://www.imdb.com'+movie+'?ref_=adv_li_tt' for movie in movies]  # link to each movie

In [15]:
Movie = pd.DataFrame([ID, links, ratings]).T
Movie.columns = ['ID','link','rating']
Movie

,ID,link,rating
0,tt0448115,https://www.imdb.com/title/tt0448115/?ref_=adv...,7.6
1,tt2274648,https://www.imdb.com/title/tt2274648/?ref_=adv...,5.4
2,tt4154664,https://www.imdb.com/title/tt4154664/?ref_=adv...,7.2
3,tt1477834,https://www.imdb.com/title/tt1477834/?ref_=adv...,7.2
4,tt5886046,https://www.imdb.com/title/tt5886046/?ref_=adv...,6.4
5,tt4154756,https://www.imdb.com/title/tt4154756/?ref_=adv...,8.5
6,tt4633694,https://www.imdb.com/title/tt4633694/?ref_=adv...,8.5
7,tt4701182,https://www.imdb.com/title/tt4701182/?ref_=adv...,6.9
8,tt1488606,https://www.imdb.com/title/tt1488606/?ref_=adv...,6.5
9,tt7734218,https://www.imdb.com/title/tt7734218/?ref_=adv...,5.9


## For each movie...

#### Get the distribution of rating

use Shazam! as an example

In [16]:
total_reviews_list = []
for movie in links: 
    r1 = requests.get(movie)
    tree1 = lxml.html.fromstring(r1.content)
    total_reviews = tree1.xpath("//div/span/a/text()")[1]
    try:
        total_reviews = int(total_reviews[:-4].replace(',',''))
        total_reviews_list.append(total_reviews)
    except:
        print(movie)
        total_reviews_list.append(' ')

https://www.imdb.com/title/tt7734218/?ref_=adv_li_tt
https://www.imdb.com/title/tt9562694/?ref_=adv_li_tt


In [17]:
total_reviews_list[-10] = 66
total_reviews_list[9] = np.nan
total_reviews_list 
Movie['total_reviews'] = total_reviews_list
Movie = Movie.dropna()
Movie

,ID,link,rating,total_reviews
0,tt0448115,https://www.imdb.com/title/tt0448115/?ref_=adv...,7.6,1095.0
1,tt2274648,https://www.imdb.com/title/tt2274648/?ref_=adv...,5.4,434.0
2,tt4154664,https://www.imdb.com/title/tt4154664/?ref_=adv...,7.2,5974.0
3,tt1477834,https://www.imdb.com/title/tt1477834/?ref_=adv...,7.2,3275.0
4,tt5886046,https://www.imdb.com/title/tt5886046/?ref_=adv...,6.4,355.0
5,tt4154756,https://www.imdb.com/title/tt4154756/?ref_=adv...,8.5,3887.0
6,tt4633694,https://www.imdb.com/title/tt4633694/?ref_=adv...,8.5,1328.0
7,tt4701182,https://www.imdb.com/title/tt4701182/?ref_=adv...,6.9,1017.0
8,tt1488606,https://www.imdb.com/title/tt1488606/?ref_=adv...,6.5,727.0
10,tt2386490,https://www.imdb.com/title/tt2386490/?ref_=adv...,7.7,500.0


In [325]:
Movie['title'] = movie_title
Movie

,ID,link,rating,total_reviews,title
0,tt0448115,https://www.imdb.com/title/tt0448115/?ref_=adv...,7.6,1095.0,Shazam!
1,tt2274648,https://www.imdb.com/title/tt2274648/?ref_=adv...,5.4,434.0,Hellboy
2,tt4154664,https://www.imdb.com/title/tt4154664/?ref_=adv...,7.2,5974.0,Captain Marvel
3,tt1477834,https://www.imdb.com/title/tt1477834/?ref_=adv...,7.2,3275.0,Aquaman
4,tt5886046,https://www.imdb.com/title/tt5886046/?ref_=adv...,6.4,355.0,Escape Room
5,tt4154756,https://www.imdb.com/title/tt4154756/?ref_=adv...,8.5,3887.0,Avengers: Infinity War
6,tt4633694,https://www.imdb.com/title/tt4633694/?ref_=adv...,8.5,1328.0,Spider-Man: Into the Spider-Verse
7,tt4701182,https://www.imdb.com/title/tt4701182/?ref_=adv...,6.9,1017.0,Bumblebee
8,tt1488606,https://www.imdb.com/title/tt1488606/?ref_=adv...,6.5,727.0,Triple Frontier
9,tt2386490,https://www.imdb.com/title/tt2386490/?ref_=adv...,7.7,500.0,How to Train Your Dragon: The Hidden World


In [326]:
Movie.to_csv('Movie.csv',index = False)

# No need to run above code again. If need Movie df, read in the csv fiel

In [206]:
Movie = pd.read_csv('Movie.csv')

In [207]:
links = list(Movie.link)
ID = list(Movie.ID)
total_reviews_list = list(Movie.total_reviews)

In [208]:
Movie.total_reviews.describe()

count      38.000000
mean     1600.894737
std      1580.348555
min        66.000000
25%       611.750000
50%      1139.500000
75%      1810.750000
max      5974.000000
Name: total_reviews, dtype: float64

In [209]:
review_links = []   # review_links contains 38 lists; each list contains links to reviews from 10 different ratings
for id in ID:
    # review_dist contains 10 items; each item is a link to reviews with a rating of i
    review_dist = ['https://www.imdb.com/title/'+ id +'/reviews?sort=helpfulnessScore&dir=desc&ratingFilter={}'.format(i) for i in range(1,11)]
    review_links.append(review_dist)

In [210]:
len(review_links),len(review_links[0])

(38, 10)

In [ ]:
# Shazam! review link = review_links[0]; Shazam! review with rating of 1 = review_links[0][0]
# 1. mul = total number of reviews with rating = i
# 2. get 50 * mul/total_reviews for this movie

In [327]:
shazam = review_links[0][0]
r2 = requests.get(shazam)
tree2 = lxml.html.fromstring(r2.content)
tree2

eles = tree2.xpath('//*[@class = "lister-item-content"]')
review_title_list = []
review_text_list = []
reviewer = tree2.xpath('//*[@class="title"]/@href')
review_er_list = [item[8:-1] for item in reviewer]
for ele in eles:
    review_title = ele.xpath('a[@class="title"]/text()')
    review_text = ele.xpath('div/div[@class = "text show-more__control"]/descendant-or-self::node()/text()')
#     review_text = ele.xpath('div/div[@class = "text show-more__control"]/text()')
    review_title_list.append(review_title)
    review_text_list.append(review_text)
review_text_list[2]
# reviewer = tree2.xpath('//*[@class="title"]/@href')
# review_er_list = [item[8:-1] for item in reviewer]
# review_er_list[0]

# len(reviewer),len(review_title_list), len(review_text_list)

["Can't believe anyone would give this movie 10/10.\nThe movie is childish to the enth degree, I know Its Shazam and it needs to be a little childish but it was like watching a preschool cartoon."]

In [328]:
movie_review_df_list = []
for movie in range(len(review_links)):
    review_cnts = []  # a list holds 10 rating counts for each movie
    title_list = []
    text_list = []
    reviewer_list = []
    
    movie_review_df = pd.DataFrame()
    for rating in range(10):
        link = review_links[movie][rating]
        r2 = requests.get(link)
        tree2 = lxml.html.fromstring(r2.content)
        
        # get total number of reviews that give "rating"
        review_cnt = tree2.xpath("//div/span/text()")[3]
        review_cnt = review_cnt[:-8]
        try: 
            review_cnt = int(review_cnt.replace(',',''))
        except:
            try:
                review_cnt = int(review_cnt)
            except:
                print(review_cnt)
                review_cnt = 0
                
        review_cnts.append(review_cnt)
#         print(review_cnt)
        
        # after get the number of reviews with rating i, get 100 * rating% review text
        # 1. get all reviews 
        eles = tree2.xpath('//*[@class = "lister-item-content"]')
        review_title_list = []
        review_text_list = []
        reviewer = tree2.xpath('//*[@class="title"]/@href')
        review_er_list = [item[8:-1] for item in reviewer]
        for ele in eles:
            review_title = ele.xpath('a[@class="title"]/text()')
            review_text = ele.xpath('div/div[@class = "text show-more__control"]/descendant-or-self::node()/text()')
#             review_text = ele.xpath('div/div[@class = "text show-more__control"]/text()')
            review_title_list.append(review_title)
            review_text_list.append(review_text)
        # build a dataframe to store review info
        df = pd.DataFrame([review_er_list,review_title_list, review_text_list]).T
        df['review_rating'] = [rating+1]*len(df)
#         print("dataframe before slicing\n", df)
        # 2. slice the dataframe to retain only percentage of the reviews if the total reviews are more than 100
        if total_reviews_list[movie] > 100:
            percent = 100*review_cnt/total_reviews_list[movie] 
#             print(percent)
            df = df.loc[:percent+1]
        #concat review info of the next "rating" by rows; should get a total of 100 rows when all 10 ratings have looped
        movie_review_df = pd.concat([movie_review_df,df],ignore_index =True)
    # store movie_review_df for each movie to a list called movie_review_df_list
    movie_review_df_list.append(movie_review_df)
#     break

#### <span style="color:red">WARNING: DO NOT DELETE THIS CELL!!!</span>

In [329]:
import copy
MOVIE_REVIEW_LIST = copy.deepcopy(movie_review_df_list)

In [228]:
# len(movie_review_df_list)  #38 checked
for i in range(38):
    print(len(movie_review_df_list[i]))

113
116
115
111
114
90
88
113
113
98
95
107
114
114
114
113
116
114
114
113
113
114
82
112
111
96
104
113
103
85
112
114
112
112
115
113
103
104


In [ ]:
# now since each movie has slightly different number of reviews

In [331]:
movie_review_df_list = copy.deepcopy(MOVIE_REVIEW_LIST)
#  save to files with name of the movie
all_review = pd.DataFrame()
for i in range(len(movie_review_df_list)):
    df = movie_review_df_list[i]
    df.columns = ['reviewer','review_title','text','review_rating']
    # write to file named with movie id
    movie_name = ID[i]
    df['movie_id'] = [movie_name]*len(df)
    df.to_csv(movie_name+'.csv',index = False)
    
    all_review = pd.concat([all_review,df],ignore_index= True)
all_review.to_csv('all_review.csv',index = False)

# ========================END=============================

In [156]:
# reviewer
review_title= tree2.xpath('//*[@class="title"]/text()')
review_title


[' The worst superhero movie of all time\n',
 " It's the movie Big With superpowers!\n",
 ' Poor storyline, too childish and boring\n',
 " All the fake reviews won't hide the truth for long.\n",
 ' Complete Waste of Money and Time\n',
 " Almost walked out of the cinema, waste of time. First time I'm rating a movie with 1 star.\n",
 ' A seriously stupid movie...\n',
 " DC's power ranger\n",
 ' The last third of this movie was directed by someone else, or an idiot\n',
 ' Um ... this movie is terrible.\n',
 ' POSITIVE REVIEWS ARE A FRAUD ON YOU!\n',
 ' OMG...what a joke..an insult!\n',
 ' Unfortunately the worst DC movies ever.\n',
 ' Beware of false reviews by DC\n',
 ' What are people thinking??\n',
 ' it had the potential good casting.. very bad movie\n',
 " I still can't believe it\n",
 ' insufferable annoying trash\n',
 ' Forgettable generic trash\n',
 ' Unclear main character\n',
 ' Wow I want my life back\n',
 ' Worst superhero movie ever!\n',
 ' Low Budget Easily Forgotten Garbage

In [203]:
tree2.xpath('//*[@class="title"]/text()/../following-sibling::div[2]/div[@class = "text show-more__control"]/text()')

["The idea of the film is essentially copied/plagiarized from Tom Hanks's 1988 hit Big, but is much worse in every possible way. The visual effects are cheap and lousy. Levi's performance is a disaster. The tones of the young Billy and the grownup Billy are not consistent. The buildup is too long, and the plot lacks credibility. The result is a mindless comedy that is neither funny nor with heart.",
 "Can't believe anyone would give this movie 10/10.\nThe movie is childish to the enth degree, I know Its Shazam and it needs to be a little childish but it was like watching a preschool cartoon.",
 "This is terrible. Absolutely terrible. It's long, not funny (all the jokes in the trailers), and at the end amazingly boring.",
 "It's take ages to present the characters, the main villain is awful and I prefer not talk about that sidekick kid Freddy, probably the most annoying character in theaters for the last decade. And my god the CGI!! The demons are ugly and badly designed.",
 "This is so

In [202]:
# review_text = tree2.xpath('//*[@class = "text show-more__control"]//text()')
    
# review_text

XPathEvalError: Invalid expression

In [163]:
review_text[5]

"This is so far the worst experience I had in a theatre. Don't waste you're time on this, this movie should have make it direct-to-DVD. Hopefully, this will be a massive flop à the box office (not even 350M WW to my opinion)"

In [198]:
len(reviewer), len(review_title), len(review_text)

(25, 25, 34)

In [129]:

for movie in range(len(review_links)):
    review_cnts = []  # a list holds 10 rating counts for each movie
    review_texts = [] # a list holds 10 lists; each list contains review text; the length of each list follow the same distribution of ratings
    for i in range(10):
        rating = review_links[movie][i]
        r2 = requests.get(rating)
        tree2 = lxml.html.fromstring(r2.content)

        review_cnt = tree2.xpath("//div/span/text()")[3]
        review_cnt = review_cnt[:-8]
        try: 
            review_cnt = int(review_cnt.replace(',',''))
        except:
            review_cnt = int(review_cnt)
        review_cnts.append(review_cnt)
        
        # after get the number of reviews with rating i, get 100 * rating% review text
        if review_cnt >= 100:
            percent = review_cnt/total_reviews_list[movie] * 100
            review_title = tree2.xpath("//div/a[@class = 'title']@href")
        else: 
            # no need to normalize, get all review texts 

66

#### Get 100* rating% reviews (except when there is less than 100 reviews in total)